In [59]:
import pandas as pd
import string
import numpy as np
import json

from tensorflow.keras.preprocessing.sequence import pad_sequences

#from keras.preprocessing.sequence import pad_sequences
from keras.layers import Embedding, LSTM, Dense, Dropout
from keras.preprocessing.text import Tokenizer
from keras.callbacks import EarlyStopping
from keras.models import Sequential
import keras.utils as ku

import tensorflow as tf
tf.random.set_seed(2)
from numpy.random import seed
seed(1)

#load all the datasets 
df1 = pd.read_csv('Data/USvideos.csv')
df2 = pd.read_csv('Data/CAvideos.csv')
df3 = pd.read_csv('Data/GBvideos.csv')

#load the datasets containing the category names
data1 = json.load(open('Data/US_category_id.json'))
data2 = json.load(open('Data/CA_category_id.json'))
data3 = json.load(open('Data/GB_category_id.json'))

In [60]:
def category_extractor(data):
    i_d = [data['items'][i]['id'] for i in range(len(data['items']))]
    title = [data['items'][i]['snippet']["title"] for i in range(len(data['items']))]
    i_d = list(map(int, i_d))
    category = zip(i_d, title)
    category = dict(category)
    return category

#create a new category column by mapping the category names to their id
df1['category_title'] = df1['category_id'].map(category_extractor(data1))
df2['category_title'] = df2['category_id'].map(category_extractor(data2))
df3['category_title'] = df3['category_id'].map(category_extractor(data3))

#join the dataframes
df = pd.concat([df1, df2, df3], ignore_index=True)

#drop rows based on duplicate videos
df = df.drop_duplicates('video_id')

#collect only titles of entertainment videos
#feel free to use any category of video that you want
entertainment = df[df['category_title'] == 'Entertainment']['title']
entertainment = entertainment.tolist()



#remove punctuations and convert text to lowercase
def clean_text(text):
    text = ''.join(e for e in text if e not in string.punctuation).lower()
    
    text = text.encode('utf8').decode('ascii', 'ignore')
    return text

corpus = [clean_text(e) for e in entertainment]

In [61]:
tokenizer = Tokenizer()
def get_sequence_of_tokens(corpus):
  #get tokens
  tokenizer.fit_on_texts(corpus)
  total_words = len(tokenizer.word_index) + 1
 
  #convert to sequence of tokens
  input_sequences = []
  for line in corpus:
      token_list = tokenizer.texts_to_sequences([line])[0]
  for i in range(1, len(token_list)):
      n_gram_sequence = token_list[:i+1]
      input_sequences.append(n_gram_sequence)
 
      return input_sequences, total_words
inp_sequences, total_words = get_sequence_of_tokens(corpus)

In [62]:
def generate_padded_sequences(input_sequences):
  max_sequence_len = max([len(x) for x in input_sequences])
  input_sequences = np.array(pad_sequences(input_sequences,  maxlen=max_sequence_len, padding='pre'))
  predictors, label = input_sequences[:,:-1], input_sequences[:, -1]
  label = ku.to_categorical(label, num_classes = total_words)
  return predictors, label, max_sequence_len
predictors, label, max_sequence_len = generate_padded_sequences(inp_sequences)

In [63]:
max_sequence_len

2

In [64]:
label.shape

(1, 13915)

In [68]:
def create_model(max_sequence_len, total_words):
  input_len = max_sequence_len -10
  model = Sequential()
 
  # Add Input Embedding Layer
  model.add(Embedding(total_words, 5, input_length=6))
 
  # Add Hidden Layer 1 — LSTM Layer
  model.add(LSTM(100))
  model.add(Dropout(0.1))
 
  # Add Output Layer
  model.add(Dense(total_words, activation='softmax'))
  model.compile(loss='categorical_crossentropy', optimizer='adam')
 
  return model
model = create_model(max_sequence_len, total_words)
model.fit(predictors, label, epochs=20, verbose='auto')

Epoch 1/20
1/1 [==============================] - 3s 3s/step - loss: 9.5408
Epoch 2/20
1/1 [==============================] - 0s 21ms/step - loss: 9.5386
Epoch 3/20
1/1 [==============================] - 0s 22ms/step - loss: 9.5363
Epoch 4/20
1/1 [==============================] - 0s 26ms/step - loss: 9.5338
Epoch 5/20
1/1 [==============================] - 0s 30ms/step - loss: 9.5316
Epoch 6/20
1/1 [==============================] - 0s 25ms/step - loss: 9.5291
Epoch 7/20
1/1 [==============================] - 0s 31ms/step - loss: 9.5267
Epoch 8/20
1/1 [==============================] - 0s 32ms/step - loss: 9.5243
Epoch 9/20
1/1 [==============================] - 0s 22ms/step - loss: 9.5217
Epoch 10/20
1/1 [==============================] - 0s 27ms/step - loss: 9.5189
Epoch 11/20
1/1 [==============================] - 0s 31ms/step - loss: 9.5165
Epoch 12/20
1/1 [==============================] - 0s 22ms/step - loss: 9.5137
Epoch 13/20
1/1 [==============================] - 0s 24ms/step

In [69]:
def generate_text(seed_text, next_words, model, max_sequence_len):
  for _ in range(next_words):
      token_list = tokenizer.texts_to_sequences([seed_text])[0]
      token_list = pad_sequences([token_list], maxlen=max_sequence_len-1,  padding='pre')
      #predicted = model.predict_classes(token_list, verbose=0)
      predicted = np.argmax(model.predict(token_list,verbose=0),axis=1)
 
  output_word = ""
  for word,index in tokenizer.word_index.items():
      if index == predicted:
          output_word = word
      break
  seed_text += '' +output_word
  return seed_text.title()

In [70]:
print(generate_text('spiderman', 5, model, max_sequence_len))

Spiderman
